## Notebook for empirical comparison of JAW and JAWS runtime in "JAWS" paper (Appendix, Table 4)

These runtime experiments were run on a 2019 MacBook Pro with a 2.3 GHz 8-Core Intel Core i9 processor and 32 GB memory

#### Prinster, A., Liu, A., & Saria, S. JAWS: Auditing Predictive Uncertainty Under Covariate Shift. In Advances in Neural Information Processing Systems. 2022.

Last updated: January 4, 2023

In [1]:
## Dependencies from RUE
%matplotlib inline

import imp
import logging
imp.reload(logging)
logging.basicConfig(level=logging.INFO)

from functools import partial

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
# import pystan
import seaborn as sns

from scipy.optimize import minimize


import pandas as pd
from sklearn.preprocessing import StandardScaler
# import numpy as np 
import scipy
import math
from tqdm import tqdm
from datetime import datetime
from datetime import date
from random import sample
from utils.JAWS_utils import *
from utils import bayesnn
from utils.IF_utils import *

# from autograd import make_jvp

# import sys
# print(sys.getrecursionlimit())


INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
## Load datasets

airfoil = pd.read_csv('./datasets/airfoil/airfoil.txt', sep = '\t', header=None)
airfoil.columns = ["Frequency","Angle","Chord","Velocity","Suction","Sound"]
X_airfoil = airfoil.iloc[:, 0:5].values
X_airfoil[:, 0] = np.log(X_airfoil[:, 0])
X_airfoil[:, 4] = np.log(X_airfoil[:, 4])
Y_airfoil = airfoil.iloc[:, 5].values
n_airfoil = len(Y_airfoil)
print("X_airfoil shape : ", X_airfoil.shape)
        
winequality_red = pd.read_csv('./datasets/wine/winequality-red.csv', sep=';')
X_wine = winequality_red.iloc[:, 0:11].values
Y_wine = winequality_red.iloc[:, 11].values
n_wine = len(Y_wine)
print("X_wine shape : ", X_wine.shape)
        
wave = pd.read_csv('./datasets/WECs_DataSet/Adelaide_Data.csv', header = None)
# wave_ids = sample(range(0, len(wave)),2000)
# X_wave = wave.iloc[wave_ids, 0:48].values
# Y_wave = wave.iloc[wave_ids, 48].values
X_wave = wave.iloc[0:2000, 0:48].values
Y_wave = wave.iloc[0:2000, 48].values
n_wave = len(Y_wave)
print("X_wave shape : ", X_wave.shape)
        
superconduct = pd.read_csv('./datasets/superconduct/train.csv')
# superconduct_ids = sample(range(0, len(superconduct)),2000)
# X_superconduct = superconduct.iloc[superconduct_ids, 0:81].values
# Y_superconduct = superconduct.iloc[superconduct_ids, 81].values
X_superconduct = superconduct.iloc[0:2000, 0:81].values
Y_superconduct = superconduct.iloc[0:2000, 81].values
n_superconduct = len(Y_superconduct)
print("X_superconduct shape : ", X_superconduct.shape)
        
# UCI Communities and Crime Data Set
# download from:
# http://archive.ics.uci.edu/ml/datasets/communities+and+crime
communities_data = np.loadtxt('./datasets/communities/communities.data',delimiter=',',dtype=str)
# remove categorical predictors
communities_data = np.delete(communities_data,np.arange(5),1)
# remove predictors with missing values
communities_data = np.delete(communities_data,\
            np.argwhere((communities_data=='?').sum(0)>0).reshape(-1),1)
communities_data = communities_data.astype(float)
X_communities = communities_data[:,:-1]
Y_communities = communities_data[:,-1]
n_communities = len(Y_communities)
print("X_communities shape : ", X_communities.shape)

X_airfoil shape :  (1503, 5)
X_wine shape :  (1599, 11)
X_wave shape :  (2000, 48)
X_superconduct shape :  (2000, 81)
X_communities shape :  (1994, 99)


In [7]:
    ## Select dataset
    # dataset = 'airfoil'

def load_dataset(dataset):
        n = 200
        train_inds = np.random.choice(eval('n_'+dataset),n,replace=False)
        # train_inds = list(range(0, n))
        test_inds = np.setdiff1d(np.arange(eval('n_'+dataset)),train_inds)

#         print("train_inds[0:10] : ", train_inds[0:10])
#         print("test_inds[0:10] : ", test_inds[0:10])

        X = eval('X_'+dataset)[train_inds]
        Y = eval('Y_'+dataset)[train_inds]
        X1 = eval('X_'+dataset)[test_inds]
        Y1 = eval('Y_'+dataset)[test_inds]

        ## Normalize data
        norm_X = InputNormalizer(X)
        norm_y = TargetNormalizer(Y)

        X = norm_X.normalize(X)
        Y = norm_y.normalize(Y)

        X1 = norm_X.normalize(X1)
        Y1 = norm_y.normalize(Y1)

        return X, Y, X1, Y1


## Example empirical runtime for JAW (Prinster et al., 2022) or Jackknife+ (Barber et al., 2021) with leave-one-out retraining

In [21]:
## Trying now with Hessian having 10.0 as dampening
print(np.random.RandomState(12345))
ntrial = 1
alpha = 0.1
damp = 0.0 ## default dampening
method_names = ['IF1-jackknife', 'IF1-jackknife-mm', 'IF1-jackknife+', 'IF1-JAWA', 
                'IF2-jackknife', 'IF2-jackknife-mm', 'IF2-jackknife+', 'IF2-JAWA',
                'IF3-jackknife', 'IF3-jackknife-mm', 'IF3-jackknife+', 'IF3-JAWA']

print("\nTime beginning : ", datetime.now())

for dataset in ['airfoil', 'wine', 'wave', 'superconduct', 'communities']: #'airfoil', 'wave', , 'communities'
    
    X2_train, y2_train, X2_test, y2_test = load_dataset(dataset)
    
    print("\n DATASET : ", dataset)

    if (dataset == 'airfoil'):
        bias = 0.85
        L2_lambda = 1
    elif (dataset == 'wine'):
        bias = 0.53
        L2_lambda = 8
    elif (dataset == 'wave'):
        bias = 0.0000925
        L2_lambda = 4
    elif (dataset in ['superconduct']):
        bias = 0.00062
        L2_lambda = 96
    elif (dataset == 'communities'):
        bias = 0.825
        L2_lambda = 64
        
    rng = np.random.RandomState(0) ## Generate random state with seed=0

    n_train, n_inputs = X2_train.shape
    n_hidden = 25

    alphas = [1.0, 1.0]
    beta = 1.0
    
    print("\nBeginning retraining for " + str(dataset) + ": ", datetime.now())
    for i in tqdm.tqdm(range(0, n_train)):



        ############# Full model

        model = bayesnn.MLP(n_inputs, n_hidden)
        init_params = model.init_params(rng)

        weights = np.ones(n_train)
        weights[i] = 0

        objective, likelihood, prior, likelihood_all = bayesnn.make_objective(model, alphas, beta, n_train, weights)

        config = bayesnn.init_sgd_config()
        config['n_epochs'] = 2000
        config['batch_size'] = 50

        params = bayesnn.train(objective, init_params, X2_train, y2_train, config, weights)
        y_hat_full = model.predict(params, X2_test)
    
    print("\nEnding retraining for " + str(dataset) + ": ", datetime.now())
    


print("\nTime of completion : ", datetime.now())

RandomState(MT19937)

Time beginning :  2022-07-21 03:33:02.385043
train_inds[0:10] :  [ 640   75  154 1247 1025   32  604  968  624  227]
test_inds[0:10] :  [ 0  1  2  3  4  5  7  8  9 10]

 DATASET :  airfoil

Beginning retraining for airfoil:  2022-07-21 03:33:02.387628


100%|█████████████████████████████████████████| 200/200 [58:39<00:00, 17.60s/it]



Ending retraining for airfoil:  2022-07-21 04:31:42.263123
train_inds[0:10] :  [  29 1202 1008  328  808  220  770  776  250 1325]
test_inds[0:10] :  [0 1 2 3 4 5 6 7 8 9]

 DATASET :  wine

Beginning retraining for wine:  2022-07-21 04:31:42.266604


100%|█████████████████████████████████████████| 200/200 [57:35<00:00, 17.28s/it]



Ending retraining for wine:  2022-07-21 05:29:17.982011
train_inds[0:10] :  [ 682 1579 1195 1825  901  259 1617  232 1679  470]
test_inds[0:10] :  [0 1 2 3 4 5 6 7 8 9]

 DATASET :  wave

Beginning retraining for wave:  2022-07-21 05:29:17.986075


...


Ending retraining for wave:  2022-07-21 06:41:12.267468
train_inds[0:10] :  [ 384  844  764 1266  920 1444   26 1194  263  344]
test_inds[0:10] :  [0 1 2 3 4 5 6 7 8 9]

 DATASET :  superconduct

Beginning retraining for superconduct:  2022-07-21 06:41:12.272935


100%|███████████████████████████████████████| 200/200 [1:17:15<00:00, 23.18s/it]



Ending retraining for superconduct:  2022-07-21 07:58:27.386353
train_inds[0:10] :  [1440  561 1231 1194 1421  664  217 1900 1120 1837]
test_inds[0:10] :  [0 1 2 3 4 5 6 7 8 9]

 DATASET :  communities

Beginning retraining for communities:  2022-07-21 07:58:27.393949


100%|███████████████████████████████████████| 200/200 [1:20:42<00:00, 24.21s/it]


Ending retraining for communities:  2022-07-21 09:19:09.535821

Time of completion :  2022-07-21 09:19:09.535976


## Example empirical runtime for JAWA (influence function Approximation of JAW to avoid retraining)
Default below is 3rd order IF approximation. See Note towards bottom of cell below on how to change between 1st, 2nd, or 3rd order IF approximation

In [8]:
## Trying now with Hessian having 10.0 as dampening
print(np.random.RandomState(12345))
ntrial = 1
alpha = 0.1
damp = 0.0 ## default dampening
method_names = ['IF1-jackknife', 'IF1-jackknife-mm', 'IF1-jackknife+', 'IF1-JAWA', 
                'IF2-jackknife', 'IF2-jackknife-mm', 'IF2-jackknife+', 'IF2-JAWA',
                'IF3-jackknife', 'IF3-jackknife-mm', 'IF3-jackknife+', 'IF3-JAWA']

print("\nTime beginning : ", datetime.now())

for dataset in ['airfoil', 'wine', 'wave', 'superconduct', 'communities']: #'airfoil', 'wave', , 'communities'
    
    X2_train, y2_train, X2_test, y2_test = load_dataset(dataset)
    
    print("\n DATASET : ", dataset)

    if (dataset == 'airfoil'):
        bias = 0.85
        L2_lambda = 1
    elif (dataset == 'wine'):
        bias = 0.53
        L2_lambda = 8
    elif (dataset == 'wave'):
        bias = 0.0000925
        L2_lambda = 4
    elif (dataset in ['superconduct']):
        bias = 0.00062
        L2_lambda = 96
    elif (dataset == 'communities'):
        bias = 0.825
        L2_lambda = 64
        
    rng = np.random.RandomState(0) ## Generate random state with seed=0

    n_train, n_inputs = X2_train.shape
    n_hidden = 25

    alphas = [1.0, 1.0]
    beta = 1.0
    
    
    rng = np.random.RandomState(0) ## Generate random state with seed=0

    n_train, n_inputs = X2_train.shape
    n_hidden = 25

    alphas = [1.0, 1.0]
    beta = 1.0

    ############# Full model

    model = bayesnn.MLP(n_inputs, n_hidden)
    init_params = model.init_params(rng)

    weights = np.ones(n_train)

    objective, likelihood, prior, likelihood_all = bayesnn.make_objective(model, alphas, beta, n_train, weights)

    config = bayesnn.init_sgd_config()
    config['n_epochs'] = 2000
    config['batch_size'] = 50

    params = bayesnn.train(objective, init_params, X2_train, y2_train, config, weights)
    y_hat_full = model.predict(params, X2_test)

    print("\nBeginning IFs for " + str(dataset) + ": ", datetime.now())

    ## Hessian
    damp = 0.0
    H = autograd.hessian(likelihood_all, 0)(params, X2_train, y2_train, weights)
    H = H + damp * np.eye(len(H))
    H_inv = np.linalg.inv(H)


    for i in tqdm.tqdm(range(0, n_train)):
        weights = np.ones(n_train)
        weights[i] = 0

        ############ IF approximations
        ''' NOTE:
            -
        '''

        ## 1st-order IF
        params_IFs_1 = EvaluateThetaIJ(1, params, H_inv, likelihood_all, X2_train, y2_train, weights)
        y_hat_IFs_1 = model.predict(params_IFs_1, X2_test)

        ## 2nd-order IF
        params_IFs_2 = EvaluateThetaIJ(2, params, H_inv, likelihood_all, X2_train, y2_train, weights)
        y_hat_IFs_2 = model.predict(params_IFs_2, X2_test)

        ## 3rd-order IF
        params_IFs_3 = EvaluateThetaIJ(3, params, H_inv, likelihood_all, X2_train, y2_train, weights)
        y_hat_IFs_3 = model.predict(params_IFs_3, X2_test)

    
    print("\nEnding IFs for " + str(dataset) + ": ", datetime.now())


print("\nTime of completion : ", datetime.now())

RandomState(MT19937)

Time beginning :  2023-01-04 23:24:21.624671

 DATASET :  airfoil

Beginning IFs for airfoil:  2023-01-04 23:24:37.444226


100%|█████████████████████████████████████████| 200/200 [00:12<00:00, 15.73it/s]



Ending IFs for airfoil:  2023-01-04 23:24:50.353394

 DATASET :  wine

Beginning IFs for wine:  2023-01-04 23:25:06.963235


100%|█████████████████████████████████████████| 200/200 [00:13<00:00, 14.52it/s]



Ending IFs for wine:  2023-01-04 23:25:21.121288

 DATASET :  wave

Beginning IFs for wave:  2023-01-04 23:25:42.073963


...


Ending IFs for wave:  2023-01-04 23:26:02.686127

 DATASET :  superconduct

Beginning IFs for superconduct:  2023-01-04 23:26:26.054970


100%|█████████████████████████████████████████| 200/200 [00:27<00:00,  7.19it/s]



Ending IFs for superconduct:  2023-01-04 23:26:57.406084

 DATASET :  communities

Beginning IFs for communities:  2023-01-04 23:27:22.023549


100%|█████████████████████████████████████████| 200/200 [00:35<00:00,  5.70it/s]


Ending IFs for communities:  2023-01-04 23:28:01.963919

Time of completion :  2023-01-04 23:28:01.964090
